In [1]:
import os
api_key = os.environ["MISTRAL_API_KEY"]
print(os.getenv("MISTRAL_API_KEY"))

n8zOo7FKHGqM8Jebj7KKTEYGtDaOwF0X


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain


In [3]:
import ollama

In [4]:
# Load data
loader = TextLoader("/Users/rianrachmanto/miniforge3/project/RAG_Drill_Report/data/well_remark.txt")
docs = loader.load()

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents = text_splitter.split_documents(docs)


In [6]:
from langchain_community.embeddings import OllamaEmbeddings

In [7]:
from langchain_community.vectorstores import FAISS

In [8]:
db = FAISS.from_documents(documents, OllamaEmbeddings(model="nomic-embed-text",show_progress=True))



OllamaEmbeddings: 100%|██████████| 5603/5603 [01:59<00:00, 46.90it/s]


In [9]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [10]:
# Define LLM
model = ChatMistralAI(mistral_api_key=api_key)
# Define prompt template
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

In [11]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "List date of running tools that encountered an obstruction, and also please include if there is any sand encountered during running"})
print(response["answer"])

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 19.01it/s]


Based on the provided context, only one date meets the criteria of running tools that encountered an obstruction:

1. 10-Jan-2013: MU shifting tool with a roller stem was run, and it stopped going in the hole at 6600' (71* deviation). They pumped the toolstring downhole and opened the SSD at 9546', then pulled the toolstring out of the hole (POOH). It is not explicitly mentioned if there was sand encountered during this operation. The other entries do not mention any obstructions during tool running.
